    

# Яндекс.Музыка

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс.Музыки вы сравните поведение пользователей двух столиц.

**Цель исследования** — проверьте три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные о поведении пользователей вы получите из файла `yandex_music_project.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Вы проверите данные на ошибки и оцените их влияние на исследование. Затем, на этапе предобработки вы поищете возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

In [2]:
# импорт библиотеки pandas
import pandas as pd
# чтение файла с данными и сохранение в df
# df = pd.read_csv('C:\Bacek\DS\project_1\yandex_music_project.csv')
df = pd.read_csv('yandex_music_project.csv')
# получение первых 10 строк таблицы df
df.head(2)

,Unnamed: 0,userID,Track,artist,genre,City,time,Day
0,0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday


In [3]:
df.pop('Unnamed: 0') #Удаление лишнего столбца
df.head(2)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday


In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Слова написаны слитно, не использован snake_case (правильнее user_id)

**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Стиль заголовков

In [5]:
# перечень названий столбцов таблицы df
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Переименовываю
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [8]:
# переименование столбцов
df = df.rename(columns={'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day':'day'})
# проверка результатов - перечень названий столбцов
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений

In [9]:
# подсчёт пропусков
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Меняем пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. 

In [10]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
for name in columns_to_replace:
    df[name]=df[name].fillna('unknown')
# подсчёт пропусков
df.isna().sum()

### Дубликаты

In [14]:
# подсчёт явных дубликатов
print(f'{df.duplicated().sum()=}\n\n')
df.info()

df.duplicated().sum()=3826


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  65079 non-null  object
 1   track    65079 non-null  object
 2   artist   65079 non-null  object
 3   genre    65079 non-null  object
 4   city     65079 non-null  object
 5   time     65079 non-null  object
 6   day      65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


In [15]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)
# проверка на отсутствие дубликатов
df.duplicated().sum()

Избавление от неявных дубликатов `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

In [17]:
# Просмотр уникальных названий жанров
unic_genres = df['genre']
unic_genres.sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

In [25]:
# Функция для замены неявных дубликатов
def replace_wrong_genres(wrong_genres, correct_genres):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genres)
# Устранение неявных дубликатов 
replace_wrong_genres(['hip', 'hop', 'hip-hop'],'hiphop')
# Проверка на неявные дубликаты
unic_genres = df['genre']
unic_genres.sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверьте это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделите пользователей Москвы и Санкт-Петербурга
* Сравните, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

In [26]:
# Подсчёт прослушиваний в каждом городе
df_city_group = df.groupby('city')
df_city_group = df_city_group['track'].count()
strannaya_i_neponyatnaya_peremennaya_kotoraya_bolshe_ne_ponadobitca = 45360/19719
display(df_city_group)
print(f'Разница в количестве прослушиваний в Москве по отношению к Питеру в {strannaya_i_neponyatnaya_peremennaya_kotoraya_bolshe_ne_ponadobitca} раза')

city
Moscow              45360
Saint-Petersburg    19719
Name: track, dtype: int64

Разница в количестве прослушиваний в Москве по отношению к Питеру в 2.3003194888178915 раза


В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируйте данные по дню недели и подсчитайте прослушивания в понедельник, среду и пятницу. Учтите, что в данных есть информация только о прослушиваниях только за эти дни.


In [27]:
# Подсчёт прослушиваний в каждый из трёх дней
df_days_group = df.groupby('day')
df_days_group = df_days_group['track'].count()
df_days_group

day
Friday       23149
Monday       22697
Wednesday    19233
Name: track, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [29]:
df.head(3)

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday


In [30]:
def number_tracks(day, city):
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count() 
    return track_list_count

In [36]:
# количество прослушиваний в Москве по понедельникам
monday_moscow_count = number_tracks('Monday','Moscow')
monday_moscow_count

16715

In [37]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
monday_spb_count = number_tracks('Monday','Saint-Petersburg')
monday_spb_count

5982

In [38]:
# количество прослушиваний в Москве по средам
wednesday_moscow_count = number_tracks('Wednesday','Moscow')
wednesday_moscow_count

11755

In [39]:
# количество прослушиваний в Санкт-Петербурге по средам
wednesday_spb_count = number_tracks('Wednesday','Saint-Petersburg')
wednesday_spb_count

7478

In [40]:
# количество прослушиваний в Москве по пятницам
friday_moscow_count = number_tracks('Friday','Moscow')
friday_moscow_count

16890

In [41]:
# количество прослушиваний в Санкт-Петербурге по пятницам
friday_spb_count = number_tracks('Friday','Saint-Petersburg')
friday_spb_count

6259

In [42]:
# Таблица с результатами
columns = ['Moscow', 'monday', 'wednesday', 'friday']
results = [['moskow', monday_moscow_count, wednesday_moscow_count, friday_moscow_count],  
           ['Saint-Petersburg', monday_spb_count, wednesday_spb_count, friday_spb_count]]
days_city_df = pd.DataFrame(data = results, columns=columns)
days_city_df

,Moscow,monday,wednesday,friday
0,moskow,16715,11755,16890
1,Saint-Petersburg,5982,7478,6259


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [48]:
moscow_general = df[df['city'] == 'Moscow']
spb_general = df[df['city'] == 'Saint-Petersburg']

In [70]:
def genre_weekday(table, day, time1, time2):
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df_count = genre_df.groupby('genre')
    genre_df_count = genre_df_count['genre'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    if time1 == '07:00':
        genre_df_sorted.name = f'{day}_morning'.lower()
    else:
        genre_df_sorted.name = f'{day}_evening'.lower()
    return genre_df_sorted.head(10)



In [71]:
genre_morning_monday_moscow = genre_weekday(moscow_general, 'Monday', '07:00', '11:00')
genre_evening_monday_moscow = genre_weekday(moscow_general, 'Monday', '17:00', '23:00')
pd.concat([genre_morning_monday_moscow, genre_evening_monday_moscow], axis=1)


,monday_morning,monday_evening
genre,,
pop,830.0,763.0
dance,589.0,552.0
rock,511.0,546.0
electronic,501.0,516.0
hiphop,311.0,247.0
ruspop,203.0,151.0
world,190.0,184.0
rusrap,188.0,144.0
alternative,175.0,191.0


In [72]:
genre_morning_monday_spb = genre_weekday(spb_general, 'Monday', '07:00', '11:00')
genre_evening_monday_spb = genre_weekday(spb_general, 'Monday', '17:00', '23:00')
pd.concat([genre_morning_monday_spb, genre_evening_monday_spb], axis=1)

,monday_morning,monday_evening
genre,,
pop,238.0,282.0
dance,192.0,199.0
rock,173.0,226.0
electronic,154.0,201.0
hiphop,88.0,117.0
ruspop,68.0,57.0
alternative,65.0,80.0
rusrap,56.0,NaN
jazz,47.0,61.0


In [73]:
genre_morning_friday_moscow = genre_weekday(moscow_general, 'Friday', '07:00', '11:00')
genre_evening_friday_moscow = genre_weekday(moscow_general, 'Friday', '17:00', '23:00')
pd.concat([genre_morning_friday_moscow, genre_evening_friday_moscow], axis=1)

,friday_morning,friday_evening
genre,,
pop,838,761
dance,614,521
rock,537,546
electronic,533,510
hiphop,291,282
ruspop,217,184
classical,211,171
world,182,220
alternative,181,176


In [74]:
genre_morning_friday_spb = genre_weekday(spb_general, 'Friday', '07:00', '11:00')
genre_evening_friday_spb = genre_weekday(spb_general, 'Friday', '17:00', '23:00')
pd.concat([genre_morning_friday_spb, genre_evening_friday_spb], axis=1)

,friday_morning,friday_evening
genre,,
pop,225.0,279.0
dance,201.0,221.0
electronic,181.0,227.0
rock,167.0,230.0
hiphop,113.0,103.0
classical,66.0,64.0
rusrap,60.0,66.0
alternative,57.0,67.0
world,51.0,60.0


**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [80]:
moscow_general.groupby(by = 'genre').count()['user_id'].sort_values(ascending=False).head(3)

genre
pop      6253
dance    4707
rock     4188
Name: user_id, dtype: int64

In [79]:
spb_general.groupby(by = 'genre').count()['user_id'].sort_values(ascending=False).head(3)


genre
pop      2597
dance    2054
rock     2004
Name: user_id, dtype: int64

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Проверено 3 гипотезы:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных. 
С методами проверок гипотез вы ещё познакомитесь в следующих темах.